In [2]:
import numpy as np
import numpy.linalg as la
import scipy.optimize as sopt
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from numpy import linalg as LA
%matplotlib qt

In [3]:
def funcx(x):
    fx = 100*(np.sqrt(x[0]**2+(x[1]+1)**2)-1)**2 + 90*(np.sqrt(x[0]**2+(x[1]+1)**2)-1)**2 -(20*x[0]+40*x[1])
    return fx

In [4]:
def gradient(x,delta):
    grad=np.zeros(2)
    grad[0]=(funcx([x[0]+delta,x[1]])- funcx([x[0]-delta,x[1]]))/(2*delta)
    grad[1]=(funcx([x[0],x[1]+delta])- funcx([x[0],x[1]-delta]))/(2*delta)
    return grad

In [5]:
def golden(x,search,xi,eps):
    a = xi[0];
    b = xi[1];
    tau = 0.381967;
    alpha1 = a*(1-tau) + b*tau;
    alpha2 = a*tau + b*(1-tau);
    falpha1 = funcx(x+alpha1*search);
    falpha2 = funcx(x+alpha2*search);
    for i in range(100):
        if falpha1 > falpha2:
            a = alpha1;
            alpha1 = alpha2;
            falpha1 = falpha2;
            alpha2 = tau*a + (1-tau)*b;
            falpha2 = funcx(x+alpha2*search);
        else:
            b = alpha2;
            alpha2 = alpha1;
            falpha2 = falpha1;
            alpha1 = tau*b + (1-tau)*a;
            falpha1 = funcx(x+alpha1*search);

        if np.abs(funcx(x+alpha1*search)- funcx(x+alpha2*search)) < eps :
            break;
    return alpha1,falpha1

In [6]:
def hessian(x,delta):
    H=np.zeros([2,2])
    H[0,0]= ( funcx([x[0]+delta,x[1]])  - 2*funcx(x) + funcx([x[0]-delta,x[1]]) )/ delta**2;
    H[1,1]= ( funcx([x[0],x[1]+delta])  - 2*funcx(x) + funcx([x[0],x[1]-delta]) )/ delta**2; 
    H[0,1]= ( funcx([x[0]+delta,x[1]+delta]) - funcx([x[0]+delta,x[1]-delta]) - funcx([x[0]-delta,x[1]+delta]) + funcx([x[0]-delta,x[1]-delta]) )/ (4*(delta**2));
    H[1,0]=H[0,1]
    return H

In [7]:
delta=1e-3
ep1=1e-3
xi = [-1,1]
x = xi
A=np.eye(len(x))
fx_prev=funcx(x)
print("Initial function value = %f \n " % fx_prev)
print("No. \t \t x-vector \t   f(x) \t Deriv \n")
print("______________________________________________________________________\n")
for j in range(2):
    if j==0:
        dire_prev=gradient(x,delta)
        si_prev = -dire_prev
        alpha,fx_prev = golden(x,si_prev,xi,ep1);
        
        if LA.norm(dire_prev)<ep1:
            break
        x_c = x +  alpha*si_prev
        print("%d\t %f %f\t %f \t%f \n" %(j,x_c[0],x_c[1],fx_prev,LA.norm(dire_prev)))
    else:
        deltax=x_c-x
        dire_c=gradient(x_c,delta)
        deltag=dire_c-dire_prev
        ter1= np.matmul(np.atleast_2d(deltax).transpose(),np.atleast_2d(deltax) ) /    np.matmul(deltax,deltag.transpose() )
        ter2= np.matmul(np.matmul(np.matmul(A, np.atleast_2d(deltag).transpose()), np.atleast_2d(deltag)),A) / np.matmul(np.matmul(np.atleast_2d(deltag),A ), np.atleast_2d(deltag).transpose())
        print(ter1)
        print(ter2)
        A = A + ter1 - ter2;
        si=np.matmul(-A,dire_c.transpose())
        si=np.ndarray.flatten(si.transpose())
        alpha,fx_curr = golden(x_c[:],si,xi,ep1)
        
        if abs(fx_curr-fx_prev)<ep1 or LA.norm(dire_c)<ep1:
            break
        fx_prev=fx_curr
        dire_prev=dire_c
        x=x_c
        x_c = x +  alpha*si
        print("%d\t %f %f\t %f \t%f \n" %(j,x_c[0],x_c[1],fx_curr,LA.norm(dire_c)))
#         si = - dire +((LA.norm(dire)**2)/(LA.norm(dire_prev)**2))*si_prev;
#         alpha,fx_curr = golden(x,si,xi,ep1);
#         if abs(fx_curr-fx_prev)<ep1 or LA.norm(dire)<ep1:
#             break;
#         dire_prev=dire
#         si_prev=si
#         x = x +  (alpha*si).transpose() 
#         fx_prev=funcx(x)
    
#     print("%d\t %f %f\t %f \t%f \n" %(j,x[0],x[1],fx_prev,LA.norm(dire)))
print("%d\t %f %f\t %f \t%f \n" %(j,x_c[0],x_c[1],fx_curr,LA.norm(dire_c)))
print("______________________________________________________________________\n")    

Initial function value = 270.294169 
 
No. 	 	 x-vector 	   f(x) 	 Deriv 

______________________________________________________________________

0	 -0.389200 -0.009202	 8.942496 	444.315788 

[[ 0.00071226 -0.00117684]
 [-0.00117684  0.00194445]]
[[ 0.20394345 -0.40292744]
 [-0.40292744  0.79605655]]
1	 0.022020 0.198927	 -0.863527 	33.677247 

1	 0.022020 0.198927	 -0.863527 	33.677247 

______________________________________________________________________

